In [1]:
import graphlab as gl

## Product Review Data

In [2]:
products = gl.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to claus@calogica.comand will expire on September 21, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-64824 - Server binary: /Users/claus/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444356652.log
[INFO] GraphLab Server Version: 1.6.1


In [ ]:
products.save("amazon_baby_products.csv", format='csv')

In [3]:
products.show()

Canvas is accessible via web browser at the URL: http://localhost:53226/index.html
Opening Canvas in default web browser.


## Wordcount vector

In [4]:
products['word_count'] = gl.text_analytics.count_words(products['review'])

In [5]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


In [6]:
gl.canvas.set_target('ipynb')

In [7]:
products['name'].show()

## Explore most popular product

In [8]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews['rating'].show(view='Categorical')

## Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

### Define positive/negative sentiment

In [12]:
# ignore all 3 start reviews
products = products[products['rating'] != 3]

In [13]:
# pos =4 or 5* review
products['sentiment'] = products['rating'] >= 4

In [14]:
products['sentiment'].show(view='Categorical')

In [15]:
train_data, test_data = products.random_split(.8, seed=0)

In [16]:
print len(train_data)
print len(test_data)

133448
33304


In [48]:
test_data['sentiment'].show(view='Categorical')

In [17]:
sentiment_model = gl.logistic_classifier.create(train_data, target='sentiment', 
                                                features=['word_count'],
                                               validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 3.338066     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 5.824301     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 6.707406     | 0.92

## Evaluate

In [21]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.217317806161 | 0.00560154131583 | 28028 | 5324 |
 | 0.0010000000475  | 0.782682193839 |  0.994398458684  | 28028 | 5324 |
 | 0.00200000009499 | 0.743050338092 |  0.993149707435  | 28028 | 5324 |
 | 0.00300000002608 | 0.721074380165 |  0.992329099472  | 28028 | 5324 |
 | 0.00400000018999 | 0.705672426747 |  0.991758241758  | 28028 | 5324 |
 | 0.00499999988824 | 0.694402704733 |  0.991330098473  | 28028 | 5324 |
 | 0.00600000005215 | 0.684635612322 |  0.990830597973  | 28028 | 5324 |
 | 0.00700000021607 | 0.67505634861  |  0.990295418867  | 28028 | 5324 |
 | 0.00800000037998 | 0.665101427498 |  0.989974311403  | 28028 | 5324 |
 | 0.00899999961257 

In [22]:
sentiment_model.show(view='Evaluation')

In [23]:
sentiment_model.evaluate(test_data, metric='accuracy')

{'accuracy': 0.916256305548883}

## Apply learned model

In [ ]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [ ]:
giraffe_reviews.head()

## Sort reviews on pred sentiment

In [ ]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [ ]:
giraffe_reviews.head()

In [ ]:
## Homework

In [32]:
products['word_count'][0];

In [33]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [34]:
def single_word_count(word_count, word):
    if word in word_count:
        return word_count[word]
    else:
        return 0

In [26]:
# test if it works:
single_word_count(products['word_count'][0], 'and')

3

In [27]:
products['awesome'] = products['word_count'].apply(lambda x: single_word_count(x, 'awesome'))

In [28]:
print '% awesome:', products['awesome'].sum()*1.00/len(products['awesome'])*1.000

% awesome: 0.0120058530033


In [35]:
for word in selected_words:
#     print word
    products[word] = products['word_count'].apply(lambda x: single_word_count(x, word))
    print '% {0} {1}: {2}'.format(word, products[word].sum(), products[word].sum()*1.00/len(products[word])*1.000)

% awesome 2002: 0.0120058530033
% great 42420: 0.254389752447
% fantastic 873: 0.00523531951641
% amazing 1305: 0.00782599309154
% love 40277: 0.241538332374
% horrible 659: 0.00395197658799
% bad 3197: 0.0191721838419
% terrible 673: 0.004035933602
% awful 345: 0.00206894070236
% wow 131: 0.00078559777394
% hate 1057: 0.00633875455767


In [36]:
train_data, test_data = products.random_split(.8, seed=0)

In [37]:
sentiment_model_selected = gl.logistic_classifier.create(train_data, target='sentiment', 
                                                features=selected_words,
                                               validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.251730     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.408706     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.568733     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [ ]:
sentiment_model_selected.evaluate(test_data, metric='roc_curve');
sentiment_model_selected.show(view='Evaluation')

In [38]:
ev = sentiment_model_selected.evaluate(test_data, metric='auto');
sentiment_model_selected.show(view='Evaluation')

In [ ]:
ev

In [39]:
sentiment_model_selected['coefficients'].sort('value', ascending=False)

name,index,class,value
love,None,1,1.39989834302
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
amazing,None,1,0.892802422508
fantastic,None,1,0.891303090304
great,None,1,0.883937894898
wow,None,1,-0.0541450123333
bad,None,1,-0.985827369929
hate,None,1,-1.40916406276
awful,None,1,-1.76469955631


In [47]:
sentiment_model_selected

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 12
Number of examples            : 133448
Number of classes             : 2
Number of feature columns     : 11
Number of unpacked features   : 11

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 6
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 1.0831

Settings
--------
Log-likelihood                : 54057.6401

Highest Positive Coefficients
-----------------------------
love                          : 1.3999
(intercept)                   : 1.3673
awesome                       : 1.058
amazing                       : 0.8928
fantastic                     : 0.8913

Lowest Negative Coefficients
----------------------------
terrible                      : -2.0905
horrible                

In [40]:
diaper_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [41]:
diaper_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_reviews, output_type='probability')

In [42]:
diaper_reviews['predicted_sentiment_selected'] = sentiment_model_selected.predict(diaper_reviews, 
                                                                                  output_type='probability')

In [43]:
top_diaper_review = diaper_reviews.sort('predicted_sentiment', ascending=False)[0]

In [44]:
diaper_reviews.sort('predicted_sentiment_selected', ascending=False);
top_diaper_review

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 0,
 'hate': 0,
 'horrible': 0,
 'love': 0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9999999372669541,
 'predicted_sentiment_selected': 0.7969408512906712,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little on

In [46]:
sentiment_model_selected[sentiment_model_selected['review'] == [top_diaper_review['review']]]

[ERROR] Toolkit error: Field 'review' does not exist. Use list_fields() for a list of fields that can be queried.



ToolkitError: Field 'review' does not exist. Use list_fields() for a list of fields that can be queried.
